In [1]:
import os

import nbimporter

import utils.file_manager as fm
import utils.pre_processing as ppc

Importing Jupyter notebook from C:\Users\Caio\Desktop\android-comments-miner\utils\file_manager.ipynb
Importing Jupyter notebook from C:\Users\Caio\Desktop\android-comments-miner\utils\pre_processing.ipynb


In [2]:
data_basedir = "C:\\Users\\Caio\\Desktop\\android-comments-miner\\data\\"

exports_basedir = os.path.join(data_basedir, "exports")
clean_exports_basedir = os.path.join(data_basedir, "clean_exports")
results_basedir = os.path.join(data_basedir, "results")
         
# Lista dos nomes dos diretórios que contém os exports
exports_directories = ["facebook", "telegram"]

# Colunas dos exports a serem consideradas
columns = ["Content", "CleanContent", "Source", "Rating"]

for directory in exports_directories:
    filename = directory + ".csv"
    
    export_dirpath = os.path.join(exports_basedir, directory)
    
    # Lê os exports do diretório
    export = fm.read_csv_directory(dirpath=export_dirpath, header=1)
    
    # Realiza o pré-processamento do conteúdo dos exports
    clean_export = ppc.clean_reviews(export=export)
    
    # Salva os exports já pré-processados em um arquivo .csv
    fm.data_to_csv(data=clean_export,\
                   dirpath=clean_exports_basedir,\
                   filename=filename)